In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_google_genai wikipedia pyppeteer certifi python-dotenv

In [ ]:
import os, getpass

from dotenv import load_dotenv

load_dotenv()

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

In [ ]:
_set_env("TAVILY_API_KEY")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from studio.research_assistant import analyst_graph, interview_graph, graph 

In [ ]:
from IPython.display import Image, display

In [ ]:
display(Image(analyst_graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
# Input
max_analysts = 2
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in analyst_graph.stream({"topic":topic,"max_analysts":max_analysts,}, thread, stream_mode="values"):
    # Review
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# Get state and look at next node
state = analyst_graph.get_state(thread)
state.next

In [ ]:
#### We now update the state as if we are the human_feedback node
analyst_graph.update_state(thread, {"human_analysts_feedback": 
                            "Add in someone from a startup to add an entrepreneur perspective"}, as_node="human_feedback")

In [ ]:
# Continue the graph execution
for event in analyst_graph.stream(None, thread, stream_mode="values"):
    # Review
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)

In [ ]:
# If we are satisfied, then we simply supply no feedback
further_feedack = None
analyst_graph.update_state(thread, {"human_analyst_feedback": 
                            further_feedack}, as_node="human_feedback")

In [ ]:
# Continue the graph execution to end
for event in analyst_graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [ ]:
final_state = analyst_graph.get_state(thread)
analysts = final_state.values.get('analysts')

In [ ]:
for analyst in analysts:
    print(f"Name: {analyst.name}")
    print(f"Affiliation: {analyst.affiliation}")
    print(f"Role: {analyst.role}")
    print(f"Description: {analyst.description}")
    print("-" * 50) 

In [ ]:
display(Image(interview_graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
from IPython.display import Markdown
from langchain_core.messages import  HumanMessage

messages = [HumanMessage(f"So you said you were writing an article on {topic}?")]
thread = {"configurable": {"thread_id": "100"}}
interview = interview_graph.invoke({"analyst": analysts[0], "messages": messages, "max_num_turns": 2}, thread)
Markdown(interview['sections'][0])

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Inputs
max_analysts = 3 
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "2"}}

# Run the graph until the first interruption
for event in graph.stream({"topic":topic,
                           "max_analysts":max_analysts}, 
                          thread, 
                          stream_mode="values"):
    
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# We now update the state as if we are the human_feedback node
graph.update_state(thread, {"human_analysts_feedback": 
                                "Add in the CEO of gen ai native startup"}, as_node="human_feedback")

In [ ]:
# Check
for event in graph.stream(None, thread, stream_mode="values"):
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)  

In [ ]:
# Confirm we are happy
graph.update_state(thread, {"human_analysts_feedback": 
                            None}, as_node="human_feedback")

In [ ]:
# Continue
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)